# Power management circuit

- Determine values of PMGMT circuit components using MFC energy data
- Estimate max number of samples that can be taken per cycle given the energy generated for each COD value

<br><img src="img/circuitschem.jpg" alt="Drawing" style="width: 500px;"/>

In [4]:
import numpy as np
import pandas as pd

### Power MGMT components

Comparator: LTC1540 <br>https://www.analog.com/media/en/technical-documentation/data-sheets/1540fb.pdf
<br>Required to allow cold-start of the device for in-field operation. 

Regulator: TPS7A02 <br>https://www.ti.com/lit/ds/symlink/tps7a02.pdf?ts=1592828671056&ref_url=https%253A%252F%252Fwww.google.com%252F
<br>Required to run the pic microcontroller at 1.8 V






Maximum and minimum rated supply voltages.

In [48]:
# max, min allowable supply voltage for regulator
Vmax = 6 
Vmin = 2.3 # lowest Vin for Vout = 1.8V 

### Energy harvested

In [85]:
# Energy output from single batch feed MFC, Joules (J) 
# @ different COD values

Ea = np.array([21.137,  # COD = 900
               18.017,  # COD = 500
               8.315,   # COD = 300
               2.415    # COD = 70
               ])

# Energy harvested, Joules (J) 
Ea *= 0.48 # apply energy harvesting efficiency
Ea

array([10.14576,  8.64816,  3.9912 ,  1.1592 ])

### Energy consumed

Energy consumed in one COD analysis cycle (1 week) 
    
$E_{out} = nE_{sa} + E_{e} + E_{sl} + E_{r} + E_{c}$

where 
<br>$n = $ number of samples
<br>$E_{sa} = $ energy sample
<br>$E_{e} = $ energy estimate
<br>$E_{sl} = $ energy sleep
<br>$E_{r} = $ energy regulator
<br>$E_{c} = $ energy comparator

#### Microcontroller operations $E_{sa}, E_{e}, E_{sl}$ 

In [86]:
# Micro-controller operations
mc_ops = ['sample', 'estimate', 'sleep']


# execution time (s) [measured empirically]
week = 7*24*60*60
exec_time = [0.1, 0.3, week]


# current [measured empirically]
current = np.array([1.18e-3, 2.85e-3, 2e-8]) 


# voltage [operting voltage of microcontroller]
voltage = 1.8 # 


# power
power = voltage * current
print(np.round(power,8))


# energy 
e_mc = power * exec_time 
print(np.round(e_mc,4))

Esa = e_mc[0]
Ee  = e_mc[1]
Esl = e_mc[2]

# # total energy, microcontroller
# Emc = sum(emc)
# Emc

[2.124e-03 5.130e-03 4.000e-08]
[0.0002 0.0015 0.0218]


#### Power MGMT operations - estimating performance from data sheet

The supply voltage to the comparator and voltage regulator is not constant. 
<br>It depends on the rate of discharge of the capacitor which depends on the resistance of the circuit, which is non-linear. 
<br>Deriving an equation for the power consumption precisely is therefore:
1. Beyond the scope of this study (we do not have a mathematical model for the dynamic behaviour of each component)
1. Of limited value since the amout of energy stored in the capacitor will vary with each cycle due to the stochasicity of the MFC behaviour. 

However, the following estimates can be made using information from the component data sheets about the relationship between voltage and current. 

###### 1.8V Regulator : TPS7A02
Quiescent current (IQ) is the current required to power the regulator's internal circuitry when the external load current is zero.
<br>Ground current (IGND) is the difference between the input and output currents, and necessarily includes the quiescent current. <br>A low ground current maximizes the LDO efficiency.

    IGND= IIN– IOUT

<table><tr><td> 
<img src='img/TPS7A02_ground.png' style="width: 400px;"> </td><td> 
<img src='img/TPS7A02_quiescent.png' style="width: 400px;"> </td><td> 
</table>
    
Assumptions:
- Ground current = :
    - 25nA @ 20nA ouput (microcontroller sleep = 20nA)
    - 10uA @ mA ouput (microcontroller estimate = 2.85mA)
    
- Voltage = `Vmax`
    




###### Comparator : LTC1540

<table><tr><td> 
<img src='img/IT_LTC1540.png' style="width: 300px;"> </td><td> 
<img src='img/IV_LTC1540.png' style="width: 300px;"> </td><td> 
</table>

Assumptions:
- Ground current = 0.3uA
- Voltage = `Vmax`

In [87]:
pm_ops = ['comparator', 'regulator']

# execution time (s) 
exec_time = week 


# current 
current = np.array([25e-9, 3e-7]) 


# voltage 
voltage = Vmax 


# power
power = voltage * current
print(np.round(power,8))


# energy 
e_mgmt = power * exec_time 
print(np.round(e_mgmt,4))

Ec = e_mgmt[0]
Er = e_mgmt[1]


# # total energy, power MGMT
# Emgmt = sum(emgmt)
# Emgmt

[1.5e-07 1.8e-06]
[0.0907 1.0886]


### Potential for energy autonomy 

Energy consumed in one COD analysis cycle (1 week) 
    
$E_{out} = nE_{sa} + E_{e} + E_{sl} + E_{r} + E_{c}$

where 
<br>$n = $ number of samples
<br>$E_{sa} = $ energy sample
<br>$E_{e} = $ energy estimate
<br>$E_{sl} = $ energy sleep
<br>$E_{r} = $ energy regulator
<br>$E_{c} = $ energy comparator

In [88]:
def calc_Eout(n): 
    return n*Esa + Ee + Esl + Er + Ec

### Size of capacitor

The minimum rated input voltage, for the regulator (TPS7A02) used to supply an output of 1.8 V was 2.3 V. Therefore the 'switch off' voltage of the microcontroller $V_{min}=$2.3 V. 

The capacitor must be able to store at least enough energy for one COD analysis cycle $E_{out}^{'}$, in addition to the energy stored at $V_{min}$, without exceeding the maximum rated voltage of the circuit components $V_{max}$=6 V:

$C >  \frac{2E_{out}}{(V_{max}^2-V_{min}^{2})}$

In [89]:
def calc_C(E_out):
    return 2 * E_out / (Vmax**2 - Vmin**2)

A 0.66 F capacitor was used as this is a widely available standard value that satisfies the inequality

In [90]:
C = 0.66

### Comparator hysterisis

#### Threshold voltages

Low

$V_{tl}=V_{min}$=2.3 V

High (when energy in capacitor, $E_{cap} = E_{out} + 0.5CV_{min}^{2}$)

$V_{th} = \sqrt{\frac{2E_{out}}{C} + V_{tl}^{2}}$

In [91]:
def calc_Vth(Eout, C, Vtl):
    return np.sqrt((2*Eout / C) + Vtl**2)   

In [92]:
# sampling interval = 2 mins
Eout = calc_Eout(5040) 
C = calc_C(Eout)
C = 0.66
Vtl = Vmin
Vth = calc_Vth(Eout, C, Vtl)
print(f' sampling interval = 2 mins \n Eout {round(Eout,3)} J \n capacitor {C} F \n Vtl {Vmin} V \n Vth {round(Vth, 3)} V', end='\n\n')

# sampling interval = 32 mins
Eout = calc_Eout(315) 
C = calc_C(Eout)
C = 0.66
Vtl = Vmin
Vth = calc_Vth(Eout, C, Vtl)
print(f' sampling interval = 2 mins \n Eout {round(Eout,3)} J \n capacitor {C} F \n Vtl {Vmin} V \n Vth {round(Vth, 3)} V', end='\n\n')

 sampling interval = 2 mins 
 Eout 2.273 J 
 capacitor 0.66 F 
 Vtl 2.3 V 
 Vth 3.49 V

 sampling interval = 2 mins 
 Eout 1.27 J 
 capacitor 0.66 F 
 Vtl 2.3 V 
 Vth 3.023 V



https://www.maximintegrated.com/en/design/technical-documents/app-notes/3/3616.html

<br><img src="img/comparator.png" alt="Drawing" style="width: 500px;"/>

Note, in this case the comparator is powered by the capacitor whose voltage is being monitored $V_{cc}\equiv V_{cap}$

Two equations for R3, choose lower value

$R_3= \frac{V_{r}}{i_3} ,V_{out}=0$ 


$R_3= \frac{(V_{cc}-V_{r})}{i_3} ,V_{out}=V_{cc}$ 
($V_{cc}\equiv V_{cap} = V_{tl}$)

In [93]:
# Step 1 : Calculate  R3

def calc_R3(Vref, ir3=0.2E-6, Vcc=5):
    "calculate resister R3 based on selected current ir3"
    
    #                                                _________
    # R3 = (Vref-Vout)/ir3, when OUT=low -->  Vref--|_________|--Vout=0
    #                                                   R3
    R3a = Vref / ir3 
    
    
    #                                                _________
    # R3 = (Vcc-Vref)/ir3, when OUT=high -->  Vref--|_________|--Vout=Vcc
    #                                                   R3
    R3b = (Vcc - Vref) / ir3 
    
    
    R3 = [r for r in [R3a, R3b] if r>0] # only +ve values
    
    return( round( min(R3), -4) )       # return minimum positive value of R3

In [94]:
# Step 2 : Decide hysteresis bandwidth

def calc_Vhb(Vth, Vtl):
    return Vth - Vtl

Calculate R1 by equating equations below, eliminating $R_2$:

$\frac{V_r}{R_2}= \frac{(V_{th}-V_r)}{R_1} - \frac{V_r}{R_3}$     
when $V_{out}=0$ 

$\frac{V_r}{R_2}=\frac{(V_{tl}-V_r)}{R_1} + \frac{(V_{cc}-V_r)}{R_3}$
<br>when $V_{out}=V_{tl}$ 

($V_{cc}\equiv V_{cap} = V_{tl}$)

In [95]:
# Step 3 : Calculate R1

def calc_R1(R3, Vhb, Vcc=5):
    return round( R3 * Vhb / Vcc, -4 )

In [96]:
# Step 4 : Choose Vth
# Vth = 5.2

Use either of the equations above to find R2

In [97]:
# Step 5 : Calculate R2
def calc_R2(R1, R3, Vref, Vth):
    return round( Vref / ( ((Vth - Vref)/R1) - (Vref/R3) ), -4)

In [98]:
# Step 5 : Verify threshold voltages

def calc_VTH(R1, R2, R3, Vref):
    return Vref * R1 * ((1/R1) + (1/R2) + (1/R3))

def calc_VTL(R1, R3, Vth, Vcc=5):
    if Vcc == Vtl:
        return Vref * (1 / (R2 * (1/R1 + 1/R3)) + 1)
    else:     
        return Vth - (R1 * Vcc / R3)

### Calculate resistors

In [99]:
# System A 
# a : comparator:LTC1540, 1.8V voltage regulator:TPS7A02

Vref = 1.18
print(Vth, Vtl)

3.022781079493759 2.3


In [100]:
# Step 1 : Calculate  R3 
R3 = calc_R3(Vref, Vcc=Vtl)
print('{:.3e}'.format(R3))

5.600e+06


In [101]:
# Step 2 : Decide hysteresis bandwidth
Vhb = calc_Vhb(Vth, Vtl)
print(Vhb)

0.722781079493759


In [102]:
# Step 3 : Calculate R1
R1 = calc_R1(R3, Vhb, Vcc=Vtl)

print('{:.3e}'.format(R1))

1.760e+06


In [103]:
# Step 5 : Calculate R2
R2 = calc_R2(R1, R3, Vref, Vth)

print('{:.6e}'.format(R2))

1.410000e+06


In [104]:
# Verify thresholds 
VTH = calc_VTH(R1, R2, R3, Vref)
VTL = calc_VTL(R1, R3, VTH, Vcc=Vtl)

print('Vth =' + '{:.3e}'.format(VTH) + '\n' + 'Vtl =' + '{:.3e}'.format(VTL))

Vth =3.024e+00
Vtl =2.301e+00


### Calculations used in journal paper (new threshold voltages)  

In [105]:
# System A 
# a : comparator:LTC1540, 1.8V voltage regulator:TPS7A02
Vref = 1.18
Vth = 5.44 
Vtl = 2.3

In [106]:
# Step 1 : Calculate  R3 
R3 = calc_R3(Vref, Vcc=Vtl)
print('{:.3e}'.format(R3))

5.600e+06


In [107]:
# Step 2 : Decide hysteresis bandwidth
Vhb = calc_Vhb(Vth, Vtl)

In [108]:
# Step 3 : Calculate R1
R1 = calc_R1(R3, Vhb, Vcc=Vtl)

print('{:.3e}'.format(R1))

7.650e+06


In [109]:
# Step 5 : Calculate R2
R2 = calc_R2(R1, R3, Vref, Vth)

print('{:.6e}'.format(R2))

3.410000e+06


In [110]:
# Verify thresholds 
VTH = calc_VTH(R1, R2, R3, Vref)
VTL = calc_VTL(R1, R3, VTH, Vcc=Vtl)

print('Vth =' + '{:.3e}'.format(VTH) + '\n' + 'Vtl =' + '{:.3e}'.format(VTL))

Vth =5.439e+00
Vtl =2.299e+00


###### <br><img src="img/Power_mgmt_derivation2.png" alt="Drawing" style="width: 300px;"/>

###### <br><img src="img/Power_mgmt_derivation.png" alt="Drawing" style="width: 2000px;"/>

###### <br><img src="img/Power_mgmt_circuit_calculations-page-001.jpg" alt="Drawing" style="width: 1000px;"/>

<br><img src="img/Power_mgmt_circuit_calculations-page-002.jpg" alt="Drawing" style="width: 1000px;"/>
<br><img src="img/Power_mgmt_circuit_calculations-page-002_.png" alt="Drawing" style="width: 1000px;"/>

<br><img src="img/Power_mgmt_circuit_calculations-page-003.jpg" alt="Drawing" style="width: 1000px;"/>

<br><img src="img/Power_mgmt_circuit_calculations-page-004.jpg" alt="Drawing" style="width: 1000px;"/>

